In [6]:
import pandas as pd
import numpy as np
import spacy
from spacy.matcher import PhraseMatcher
from spacy.tokens import Span
from spacy import displacy
import json
import random
import re

In [7]:

df_position_sentiment = pd.read_csv(r'./data/qb/processed/combine_aspect_position_0325_clean.csv')

In [8]:
df_position_sentiment['Asp Sentiment'].replace('0', np.nan, inplace=True)

In [9]:
df_position_sentiment.dropna(subset=['Asp Sentiment'], inplace=True)

In [10]:
#df_position_sentiment = df_position_sentiment.head(3000)
#(df_position_sentiment['Doc Sentiment'].values != df_position_sentiment['Asp Sentiment'].values).sum() 

In [11]:
#df_position_sentiment.drop(
#    df_position_sentiment[df_position_sentiment['Doc Sentiment'].values == df_position_sentiment['Asp Sentiment']
#                          .values].index, inplace=True)

In [12]:
df_position_sentiment.head()

,Unnamed: 0,Unnamed: 0.1,Review ID,Country,Version,Rating,Date,Doc Sentiment,Asp Sentiment,Review,Aspects,AspectText,Positions
0,0,0,5464235433,Canada,20.01.5,1,1/30/20,negative,neutral,latest app update . this update is terrible ba...,app,app,"7,9"
1,1,1,5464235433,Canada,20.01.5,1,1/30/20,negative,negative,latest app update . this update is terrible ba...,updates,update,"11,16"
2,2,2,5466985498,USA,20.01.5,5,1/30/20,positive,positive,life saver . one my favorite apps to use while...,app,apps,"28,31"
3,3,3,5460552711,USA,20.01.4,1,1/29/20,negative,negative,worst update yet . just deleted . crashed . di...,updates,update,"6,11"
4,4,4,5460552711,USA,20.01.4,1,1/29/20,negative,negative,worst update yet . just deleted . crashed . di...,experience,deleted,"23,29"


In [13]:
#df_position_sentiment[df_position_sentiment.AspectText == ]
print(len(df_position_sentiment))
df_position_sentiment.dropna(subset=['AspectText'], inplace=True)
print(len(df_position_sentiment))

14824
14821


In [14]:
df_position_sentiment['Asp Sentiment'].unique()

array(['neutral', 'negative', 'positive'], dtype=object)

In [15]:
#df_position_sentiment[df_position_sentiment.AspectText == ]
print(len(df_position_sentiment))
df_position_sentiment.dropna(subset=['Positions'], inplace=True)
print(len(df_position_sentiment))

14821
14821


In [16]:
df_positive = df_position_sentiment[df_position_sentiment['Rating'] == 5.0]

In [17]:
len(df_positive)

6778

In [18]:
df_negative = df_position_sentiment[df_position_sentiment['Rating'] == 1.0]

In [19]:
len(df_negative)

4657

In [20]:
(df_positive['Review'].str.len() < 250).sum()

5839

In [21]:
positive_idx = []

for index, row in df_positive.iterrows():
    positive_idx.append(index)

In [22]:
positive_idx

[2,
 6,
 12,
 49,
 52,
 53,
 54,
 57,
 58,
 67,
 68,
 77,
 78,
 81,
 89,
 90,
 105,
 106,
 118,
 119,
 125,
 127,
 137,
 138,
 139,
 140,
 141,
 144,
 146,
 177,
 205,
 240,
 245,
 246,
 248,
 261,
 262,
 269,
 270,
 272,
 273,
 274,
 275,
 276,
 277,
 278,
 279,
 282,
 293,
 294,
 296,
 306,
 310,
 317,
 319,
 337,
 344,
 348,
 349,
 350,
 354,
 364,
 374,
 385,
 386,
 389,
 405,
 406,
 441,
 457,
 458,
 465,
 480,
 494,
 495,
 501,
 512,
 513,
 514,
 515,
 523,
 601,
 605,
 607,
 608,
 611,
 637,
 648,
 649,
 650,
 651,
 662,
 669,
 670,
 671,
 672,
 678,
 689,
 691,
 692,
 693,
 697,
 702,
 710,
 711,
 712,
 720,
 722,
 723,
 737,
 747,
 751,
 752,
 756,
 769,
 774,
 775,
 776,
 777,
 778,
 782,
 791,
 792,
 796,
 808,
 838,
 847,
 848,
 849,
 869,
 870,
 878,
 886,
 887,
 921,
 931,
 937,
 938,
 939,
 940,
 941,
 950,
 956,
 958,
 964,
 972,
 977,
 980,
 981,
 989,
 990,
 991,
 992,
 993,
 1011,
 1012,
 1025,
 1026,
 1033,
 1034,
 1051,
 1059,
 1060,
 1063,
 1064,
 1070,
 1071,
 10

In [23]:
positive_aspect = df_positive.loc[positive_idx[5], :]['Asp Sentiment']
positive_aspect

'neutral'

In [24]:
#generate aspect position index

df_mix = pd.DataFrame(columns=['Review ID', 
                                       'Country', 'Version', 
                                       'Rating', 'Date', 'Doc Sentiment', 'Asp Sentiment', 
                                       'Review', 'Aspects', 'AspectText', 'Positions'])

idx_counter = 0

for index, row in df_negative.iterrows():

    positive_aspect = df_positive.loc[positive_idx[idx_counter], :]['Asp Sentiment']
    review = str(df_positive.loc[positive_idx[idx_counter], :]['Review']) + row['Review']
    
    start_idx = review.find(str(df_positive.loc[positive_idx[idx_counter], :]['AspectText']))
          
    position = str(start_idx) + ',' + str(start_idx + len(df_positive.loc[positive_idx[idx_counter], :]['AspectText']) - 1)
    
    #positive
    df_mix = df_mix.append({
        'Review ID': row['Review ID'], 
        'Country': row['Country'], 
        'Version': row['Version'],
        'Rating': row['Rating'],
        'Date': row['Date'],            
        'Doc Sentiment': row['Doc Sentiment'],
        'Asp Sentiment' : row['Asp Sentiment'],
        'Review': review,
        'AspectText': df_positive.loc[positive_idx[idx_counter], :]['AspectText'],
        'Aspects': df_positive.loc[positive_idx[idx_counter], :]['Aspects'],
        'Positions': position
    }, ignore_index=True)
    
    start_idx = review.find(row['AspectText'])          
    position = str(start_idx) + ',' + str(start_idx + len(row['AspectText']) - 1)
    
    df_mix = df_mix.append({
        'Review ID': row['Review ID'], 
        'Country': row['Country'], 
        'Version': row['Version'],
        'Rating': row['Rating'],
        'Date': row['Date'],            
        'Doc Sentiment': row['Doc Sentiment'],
        'Asp Sentiment' : row['Asp Sentiment'],
        'Review': review,
        'AspectText': row['AspectText'],
        'Aspects': row['Aspects'],
        'Positions': position
    }, ignore_index=True)
    
    idx_counter += 1

In [25]:
df_mix.Review.map(len).max()

948

In [26]:
(df_mix['Review'].str.len() < 500).sum()

7314

In [27]:
len(df_mix)

9314

In [28]:
#recalculate position
#for index, row in df_position_sentiment.iterrows():
    
#    df_position_sentiment.loc[index,'Review'] = row['Review'].replace('/', '').replace('-', '')
    
#    start_idx = row['Review'].find(row['AspectText'])
          
#    df_position_sentiment.loc[index,'Positions'] = str(start_idx) + ',' + str(start_idx + len(row['AspectText']) - 1)
    

In [29]:
df_mix.head()

,Review ID,Country,Version,Rating,Date,Doc Sentiment,Asp Sentiment,Review,Aspects,AspectText,Positions
0,5464235433,Canada,20.01.5,1,1/30/20,negative,neutral,life saver . one my favorite apps to use while...,app,apps,"29,32"
1,5464235433,Canada,20.01.5,1,1/30/20,negative,neutral,life saver . one my favorite apps to use while...,app,app,"29,31"
2,5464235433,Canada,20.01.5,1,1/30/20,negative,negative,great app for small business owner ! . i have ...,app,app,"6,8"
3,5464235433,Canada,20.01.5,1,1/30/20,negative,negative,great app for small business owner ! . i have ...,updates,update,"161,166"
4,5460552711,USA,20.01.4,1,1/29/20,negative,negative,"it just work . see above . in the office , on ...",app,software,"196,203"


In [30]:
#split data in - train test and dev
def train_validate_test_split(df, train_percent=.6, validate_percent=.2, seed=None):
    np.random.seed(seed)
    
    perm = np.random.permutation(np.arange(len(df)))
    m = len(df)
    
    train_end = int(train_percent * m)
    validate_end = int(validate_percent * m) + train_end
    train = df.iloc[perm[:train_end - 1] - 1]
    validate = df.iloc[perm[train_end:validate_end - 1] - 1]
    test = df.iloc[perm[validate_end:]]
    return train, validate, test

In [31]:
train, validate, test = train_validate_test_split(df_mix)

In [32]:
train.head()

,Review ID,Country,Version,Rating,Date,Doc Sentiment,Asp Sentiment,Review,Aspects,AspectText,Positions
7414,gp:AOqpTOH46yq5mj5I1tlqxosiTqFrm0Bx_kNzQoSVfxN...,English,NaN,1,5/6/18,negative,negative,". love the app , have used it for year . just ...",app,app,"11,13"
7772,gp:AOqpTOHSBKo2IRispw4nw3T0SzcVQ0wKa_EiYYYqLJk...,English,NaN,1,6/16/17,neutral,neutral,. i wa finally able to utilize this valuable r...,estimates,estimates,"-1,7"
4785,945572453,USA,1.3.1,1,2/18/14,negative,negative,"excellent app . keep all my invoice , estimate...",app,app,"10,12"
9063,gp:AOqpTOF2zeM9Ff1HQz40IEHUGfY7y1M733dNe-s44z1...,English,3.6.1,1,8/28/14,negative,negative,". great app , make paperless billing a breeze ...",experience,payroll,"79,85"
4461,1204509250,Canada,4.3,1,5/30/15,negative,negative,effective . efficient . essential.. began usin...,updates,updates,"-1,5"


In [33]:
len(train)

5587

### Generate files

In [34]:
def generate_input_files(d_type, data):

    #save reviews
    df_review = data[['Review']]
    df_review.to_csv(r'data/qb/{}/review.txt'.format(d_type), header=None, index=None, sep='\n', mode='a')

    #save aspects
    df_review = data[['AspectText']]
    df_review.to_csv(r'data/qb/{}/term.txt'.format(d_type), header=None, index=None, sep='\n', mode='a')

    #save positions
    df_review = data[['Positions']]
    df_review.to_csv(r'data/qb/{}/position.txt'.format(d_type), header=None, index=None, sep='\n', mode='a')

    #save label
    df_review = data[['Asp Sentiment']]
    df_review.to_csv(r'data/qb/{}/label.txt'.format(d_type), header=None, index=None, sep='\n', mode='a')

In [35]:
generate_input_files('train', train)
generate_input_files('test', test)
generate_input_files('dev', validate)